In [1]:
#import SparkSession
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('binary_class').getOrCreate()

In [2]:
#read the dataset
df=spark.read.csv('classification_data.csv',inferSchema=True,header=True)

In [ ]:
#check the shape of the data 
print((df.count(),len(df.columns)))

In [ ]:
#printSchema
df.printSchema()

In [ ]:
#number of columns in dataset
df.columns

In [ ]:
#view the dataset
df.show(5)

In [ ]:
#Exploratory Data Analysis
df.describe().show()


In [ ]:
df.groupBy('label').count().show()

In [ ]:
df.groupBy('loan_purpose').count().show()

In [ ]:
#converting categorical data to numerical form

In [3]:
#import required libraries
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler



In [4]:
loan_purpose_indexer = StringIndexer(inputCol="loan_purpose", outputCol="loan_index").fit(df)
df = loan_purpose_indexer.transform(df)
loan_encoder = OneHotEncoder(inputCol="loan_index", outputCol="loan_purpose_vec")
df = loan_encoder.transform(df)

In [ ]:
df.select(['loan_purpose','loan_index','loan_purpose_vec']).show(3,False)

In [5]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

In [6]:
df_assembler = VectorAssembler(inputCols=['is_first_loan',
 'total_credit_card_limit',
 'avg_percentage_credit_card_limit_used_last_year',
 'saving_amount',
 'checking_amount',
 'is_employed',
 'yearly_salary',
 'age',
 'dependent_number',
 'loan_purpose_vec'], outputCol="features")
df = df_assembler.transform(df)

In [ ]:
df.printSchema()

In [ ]:
df.select(['features','label']).show(10,False)

In [7]:
#select data for building model
model_df=df.select(['features','label'])

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [8]:
#split the data 
training_df,test_df=model_df.randomSplit([0.75,0.25])

In [ ]:
training_df.count()

In [ ]:
training_df.groupBy('label').count().show()

In [ ]:
test_df.count()

In [ ]:
test_df.groupBy('label').count().show()

In [ ]:
log_reg=LogisticRegression().fit(training_df)

In [ ]:
#Training Results

In [ ]:
lr_summary=log_reg.summary

In [ ]:
lr_summary.accuracy

In [ ]:
lr_summary.areaUnderROC

In [ ]:
print(lr_summary.precisionByLabel)

In [ ]:
print(lr_summary.recallByLabel)

In [ ]:
predictions = log_reg.transform(test_df)
predictions.show(10)


In [ ]:
model_predictions = log_reg.transform(test_df)


In [ ]:
model_predictions = log_reg.evaluate(test_df)


In [ ]:
model_predictions.accuracy

In [ ]:
model_predictions.weightedPrecision

In [ ]:
model_predictions.recallByLabel

In [ ]:
print(model_predictions.precisionByLabel)

In [ ]:
model_predictions.areaUnderROC

In [9]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(training_df)


In [10]:
model_predictions = rf_model.transform(test_df)


In [11]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()

rf = RandomForestClassifier()
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5,10,20,25,30])
             .addGrid(rf.maxBins, [20,30,40 ])
             .addGrid(rf.numTrees, [5, 20,50])
             .build())
cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)
cv_model = cv.fit(training_df)

In [12]:
best_rf_model = cv_model.bestModel

In [13]:
# Generate predictions for entire dataset
model_predictions = best_rf_model.transform(test_df)

In [14]:
true_pos=model_predictions.filter(model_predictions['label']==1).filter(model_predictions['prediction']==1).count()
actual_pos=model_predictions.filter(model_predictions['label']==1).count()
pred_pos=model_predictions.filter(model_predictions['prediction']==1).count()

In [15]:
#Recall 
float(true_pos)/(actual_pos)

0.912426614481409

In [16]:
#Precision on test Data 
float(true_pos)/(pred_pos)

0.8562901744719926